In [1]:
import os
import json
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import torch
os.chdir("../")
os.getcwd()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/workspace'

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [57]:
original_json = json.load(open("data/COCO/Annotations/coco_img_info.json"))
output_original_json =[]
for i in list(original_json.keys()):
    try:
        image_path = os.path.join("data/COCO/Images", original_json[i]['file_name'])
        for caption in original_json[i]['captions']:
            output_original_json.append({'caption':caption, 'image':image_path, 'image_id':"coco_"+str(i)})
    except:
        print("error")
        continue
# dump ouput_original_json
with open("data/COCO/Annotations/coco_new.json", "w") as f:
    json.dump(output_original_json, f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/COCO/Annotations/coco_img_info.json'

In [3]:
original_json = json.load(open("data/COCO/Annotations/coco_new.json"))
pre_mixgen_json = json.load(open("data/COCO/Annotations/pre_mixgen.json"))
pre_romixgen_json = json.load(open("data/COCO/Annotations/pre_romix_caption_0.01_bbox_center_0.7.json"))

# just use the 10000 image (5 captions each)
pre_mixgen_json = pre_mixgen_json[:50000]
pre_romixgen_json = pre_romixgen_json[:50000]
original_json = original_json[:50000]
len(original_json), len(pre_mixgen_json), len(pre_romixgen_json)

(50000, 50000, 50000)

In [4]:
original_coco_score = []
pre_mixgen_score = []
pre_romixgen_score = []

with open("Experiments/log.txt", "w") as f:
    for i in range(50000):
        # original
        text = original_json[i]['caption']
        image = Image.open(original_json[i]['image']).convert('RGB')
        inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        original_coco_score.append(logits_per_image.detach().cpu().numpy()[0][0])

        # mixgen
        text = pre_mixgen_json[i]['caption']
        image = Image.open(pre_mixgen_json[i]['image']).convert('RGB')
        inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        pre_mixgen_score.append(logits_per_image.detach().cpu().numpy()[0][0])

        # romixgen
        text = pre_romixgen_json[i]['caption']
        image = Image.open(pre_romixgen_json[i]['image']).convert('RGB')
        inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        pre_romixgen_score.append(logits_per_image.detach().cpu().numpy()[0][0])

        if (i + 1) % 100 == 0:
            avg_original_coco_score = sum(original_coco_score) / len(original_coco_score)
            avg_mixgen_score = sum(pre_mixgen_score) / len(pre_mixgen_score)
            avg_romixgen_score = sum(pre_romixgen_score) / len(pre_romixgen_score)
            log_string = f"Images {i+1}:\n" \
                         f"Average original score: {avg_original_coco_score:.4f}\n" \
                         f"Average pre_mixgen score: {avg_mixgen_score:.4f}\n" \
                         f"Average pre_romixgen score: {avg_romixgen_score:.4f}\n\n"
            f.write(log_string)
            print(log_string)

Images 100:
Average original score: 30.1465
Average pre_mixgen score: 27.9086
Average pre_romixgen score: 26.3408


Images 200:
Average original score: 30.3373
Average pre_mixgen score: 27.0348
Average pre_romixgen score: 26.4492


Images 300:
Average original score: 30.7167
Average pre_mixgen score: 27.1041
Average pre_romixgen score: 26.5070


Images 400:
Average original score: 30.6433
Average pre_mixgen score: 27.3956
Average pre_romixgen score: 26.7517


Images 500:
Average original score: 30.7654
Average pre_mixgen score: 27.4440
Average pre_romixgen score: 26.9166


Images 600:
Average original score: 30.9070
Average pre_mixgen score: 27.5695
Average pre_romixgen score: 27.0079


Images 700:
Average original score: 30.8291
Average pre_mixgen score: 27.7041
Average pre_romixgen score: 26.8658


Images 800:
Average original score: 30.9087
Average pre_mixgen score: 27.7739
Average pre_romixgen score: 26.9932


Images 900:
Average original score: 30.8854
Average pre_mixgen score: 27

Token indices sequence length is longer than the specified maximum sequence length for this model (221 > 77). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (221) must match the size of tensor b (77) at non-singleton dimension 1

In [9]:
# save the score
with open("Experiments/score.txt", "w") as f:
    # write average score
    avg_original_coco_score = sum(original_coco_score) / len(original_coco_score)
    avg_mixgen_score = sum(pre_mixgen_score) / len(pre_mixgen_score)
    avg_romixgen_score = sum(pre_romixgen_score) / len(pre_romixgen_score)
    log_string = f"Average original score: {avg_original_coco_score:.4f}\n" \
                    f"Average pre_mixgen score: {avg_mixgen_score:.4f}\n" \
                    f"Average pre_romixgen score: {avg_romixgen_score:.4f}\n\n"
    f.write(log_string)
    print(log_string)

Average original score: 30.4887
Average pre_mixgen score: 28.0578
Average pre_romixgen score: 27.0522




In [16]:
import pandas as pd
from tokenizers import BertWordPieceTokenizer


In [18]:
how_to_tokenize = BertWordPieceTokenizer 

In [19]:
result = tokenizer.tokenize("하이브의 여자 아이돌 그룹인 르세라핌의 복귀 날짜는 5월 1일이다.")

AttributeError: 'BertWordPieceTokenizer' object has no attribute 'tokenize'

In [20]:
result

['ᄒ',
 '##ᅡ',
 '##ᄋ',
 '##ᅵ',
 '##ᄇ',
 '##ᅳ',
 '##ᄋ',
 '##ᅴ',
 'ᄋ',
 '##ᅧ',
 '##ᄌ',
 '##ᅡ',
 'ᄋ',
 '##ᅡ',
 '##ᄋ',
 '##ᅵ',
 '##ᄃ',
 '##ᅩ',
 '##ᆯ',
 'ᄀ',
 '##ᅳ',
 '##ᄅ',
 '##ᅮ',
 '##ᆸ',
 '##ᄋ',
 '##ᅵ',
 '##ᆫ',
 'ᄅ',
 '##ᅳ',
 '##ᄉ',
 '##ᅦ',
 '##ᄅ',
 '##ᅡ',
 '##ᄑ',
 '##ᅵ',
 '##ᆷ',
 '##ᄋ',
 '##ᅴ',
 '[UNK]',
 '[UNK]',
 '5',
 '##ᄋ',
 '##ᅯ',
 '##ᆯ',
 '1',
 '##ᄋ',
 '##ᅵ',
 '##ᆯ',
 '##ᄋ',
 '##ᅵ',
 '##ᄃ',
 '##ᅡ',
 '.']